In [4]:
import pandas as pd
import numpy as np
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [6]:
CSV_PATH = "/content/NEW_asl_train_dataset.csv"
df = pd.read_csv(CSV_PATH)

print("Dataset loaded. Shape:", df.shape)
print(df.head())

keypoint_columns = df.columns[4:]

Dataset loaded. Shape: (1819, 25)
                         image_id  image_h  image_w label       WRIST  \
0  hand1_0_bot_seg_1_cropped.jpeg      400      400     0  [0.0, 0.0]   
1  hand1_0_bot_seg_2_cropped.jpeg      400      400     0  [0.0, 0.0]   
2  hand1_0_bot_seg_4_cropped.jpeg      400      400     0  [0.0, 0.0]   
3  hand1_0_bot_seg_5_cropped.jpeg      400      400     0  [0.0, 0.0]   
4  hand1_0_dif_seg_1_cropped.jpeg      400      400     0  [0.0, 0.0]   

                                     THUMB_CMC  \
0   [0.1447562873363495, -0.24095259606838226]   
1    [0.1905459612607956, 0.08959610015153885]   
2    [0.12147937715053558, 0.0712842345237732]   
3  [0.36612647771835327, -0.10619818419218063]   
4     [0.4842568039894104, -0.241292804479599]   

                                     THUMB_MCP  \
0    [0.2936331033706665, -0.5108685493469238]   
1  [0.46557456254959106, -0.05129079520702362]   
2  [0.19885818660259247, 0.049099504947662354]   
3     [0.6279745101928711,

In [7]:
def normalize_keypoints(keypoints):
    """
    keypoints: np.array shape (21, 2)
    returns: normalized keypoints as 1D vector (42,)
    """
    keypoints = np.array(keypoints, dtype=np.float32)
    wrist = keypoints[0]
    keypoints -= wrist

    max_val = np.abs(keypoints).max()
    if max_val > 0:
        keypoints /= max_val
    return keypoints.flatten()

def parse_keypoint(kp_str):
    return ast.literal_eval(kp_str)

features = []
for idx, row in df.iterrows():
    kp_list = [parse_keypoint(row[col]) for col in keypoint_columns]
    norm_kp = normalize_keypoints(kp_list)
    features.append(norm_kp)

features = np.array(features, dtype=np.float32)
print("Features shape after normalization:", features.shape)
print("First sample features:", features[0])


le = LabelEncoder()
labels = le.fit_transform(df['label'].values)

print("First 10 encoded labels:", labels[:10])
print("Classes:", le.classes_)

Features shape after normalization: (1819, 42)
First sample features: [ 0.          0.          0.13830592 -0.23021571  0.28054878 -0.48810416
  0.4690425  -0.6432188   0.666013   -0.7389616   0.02712212 -0.9185938
  0.5105348  -0.92765665  0.7792712  -0.79570127  0.9460401  -0.6878389
  0.         -0.95543987  0.5646302  -0.97636425  0.82932293 -0.8304167
  0.9951982  -0.688486    0.03251593 -0.90062493  0.59169567 -0.92159086
  0.84894794 -0.7735213   1.         -0.6315396   0.10904135 -0.7791894
  0.5780015  -0.7583296   0.7970938  -0.65965086  0.92574894 -0.56676745]
First 10 encoded labels: [0 0 0 0 0 0 0 0 0 0]
Classes: ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h'
 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [8]:
class KeypointDataset(Dataset):
    def __init__(self, features, labels):
        self.X = torch.tensor(features)
        self.y = torch.tensor(labels)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = KeypointDataset(features, labels)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Train samples:", len(train_dataset), "Test samples:", len(test_dataset))

Train samples: 1637 Test samples: 182


In [9]:
class ASLMLP(nn.Module):
    def __init__(self, input_size=42, num_classes=36):
        super(ASLMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.model(x)

model = ASLMLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30

In [10]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    print(f"\n=== Epoch {epoch+1}/{num_epochs} ===")

    for i, (X_batch, y_batch) in enumerate(tqdm(train_loader, desc="Training batches")):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

        if i % 10 == 0:
            print(f"Batch {i} | Loss: {loss.item():.4f}")

    train_loss = running_loss / total
    train_acc = correct / total * 100
    print(f"Epoch {epoch+1} summary | Loss: {train_loss:.4f} | Accuracy: {train_acc:.2f}%")


=== Epoch 1/30 ===


Training batches: 100%|██████████| 52/52 [00:00<00:00, 75.49it/s] 


Batch 0 | Loss: 3.5675
Batch 10 | Loss: 3.5530
Batch 20 | Loss: 3.6000
Batch 30 | Loss: 3.5692
Batch 40 | Loss: 3.5318
Batch 50 | Loss: 3.5520
Epoch 1 summary | Loss: 3.5580 | Accuracy: 4.58%

=== Epoch 2/30 ===


Training batches:   0%|          | 0/52 [00:00<?, ?it/s]

Batch 0 | Loss: 3.4947
Batch 10 | Loss: 3.4317
Batch 20 | Loss: 3.4048
Batch 30 | Loss: 3.3754


...

Batch 0 | Loss: 0.5464
Batch 10 | Loss: 0.3182
Batch 20 | Loss: 0.6118
Batch 30 | Loss: 0.6598


Training batches:  79%|███████▉  | 41/52 [00:00<00:00, 403.09it/s]

Batch 40 | Loss: 0.4163


Training batches: 100%|██████████| 52/52 [00:00<00:00, 402.53it/s]

Batch 50 | Loss: 0.3887
Epoch 30 summary | Loss: 0.5315 | Accuracy: 80.21%


In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

test_acc = correct / total * 100
print(f"\nTest Accuracy: {test_acc:.2f}%")


Test Accuracy: 84.07%


In [12]:
MODEL_PATH = "asl_mlp_model.pth"
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved to", MODEL_PATH)

Model saved to asl_mlp_model.pth
